In [333]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math 
from functools import reduce


def ecdf(data):
    
    n = len(data)
    
    x = np.sort(data)
    y = np.arange(1,n+1)/n
    
    return x,y

def combination(n, r):
    
    if n - r == 0:
        return 1
    elif r == 0:
        return 1
    
    numerator = reduce(lambda x,y: x*y,range(n,1,-1))
    denominator = reduce(lambda x,y: x*y, range(r,0,-1))
    denominator *= reduce(lambda x,y:x*y, range(1, n-r+1)) 
    
    return numerator//(denominator)

def binomialDistribution(trials,outcomes,probability):
        
    combinations = combination(trials, outcomes)
    success_p = np.power(probability, outcomes)
    fail_p = np.power(1-probability, trials-outcomes)
    
    return combinations*success_p*fail_p

def cbdp(trials,outcomes,probability):
    """ This method calculates the Cumulative binomial distribution probability """
    
    total_probability = 1- reduce(lambda x,y: x+binomialDistribution(trials,y,probability), range(0,outcomes))
        
    return total_probability

def successChance(nAttacker, threshold,probability, multiplier=1):
    """Calculates the cbdp and sums all the probabilities up to the threshold value"""
    hits = 0
    cProbability = 0
        
    for i in range(nAttacker*multiplier,0,-1):
        cProbability = cbdp(nAttacker*multiplier,i,probability)
        hits = i
        if cProbability >= threshold:
            break
    if cProbability == 1:
        print("Probability of success"+str(probability))
        print("Hits"+str(hits))
        cProbability = binomialDistribution(nAttacker*multiplier,hits,probability)
        print("Current Probability"+str(cProbability))
    return (hits,cProbability)
    

def enemyUnitsEliminatedByAttackerUnitWithMelee(nAttacker, wsbs, strength,wounds,
                                       toughness, armor, invulnerable=0, ap=0,
                                       attacks=1, damage=1, modifiers = None,
                                       fnp = False, spreadDamage = False, threshold=0.9,
                                       toHitReRoll=False,toWoundReRoll=False):
    """This method calculates how many units will be eliminated according to the following parameters:
        Input
        nAttacker = quantity of models attacking the enemy unit
        wsbs = Weapon skill or Ballistic Skill of the attacker unit
        strength = Strength of the attacking unit 
        wounds = wounds per model in the enemy unit (Not a sum, just the wound characteristic of the enemy unit)
        toughness = Toughness of the enemy unit
        armor = Armor save of the enemy unit
        invulnerable = Invulnerable save of the enemy unit
        ap = Armor penetration characteristic of the weapon used by the attacking unit. Always use positive numbers.
        attacks = numer of attacks per model in the attacking unit. The number of attacks will be used to multiply
        by the nAttacker parameter to determine the total hits per the whole unit
        damage = Damage of the weapon being used by the attacking unit
        modifiers = Still in progress
        fnp = Additional save for the enemy unit to withstand wounds. Still in progress
        spreadDamage = Allows to spread the damage characteristic of the weapon used and calculate the amount of kills
        to the enemy unit. In other words, weapon damage will not be discarded when calculating killings.
        threshold = Expected probability for the amount of killings. For example, if you want to determine how many kills
        you will achieve with a probability p, i.e. 90%, with a unit of 10 bloodletters with S5, 2 Attacks (Unstopabble Ferocity)
        AP -3 to a Interceptor Squad , you use the threshold value of 0.9
        toHitReRoll = Still in progress. Used to determine all the hits even when rerolls occur
        toWoundReRoll = Still in progress. Used to determine all the wounds even when rerolls occur
        OUtput
        kills = amount of killings. You can expect this value to be final, so even if the enemy resisted 3 wounds
        and kills is 5 you actually would kill five
        pHit = Cumulative probability to hit 
        pWound = Cumulative probability to wound
        prWound = Cumulative probability to wound
        rWound = Amount of wounds resisted before calculated kills
    """
    
    
    if modifiers  == None:
        
        toHit_P = 1-((wsbs-1)/6)
        toWound_P = 0
        if strength*2<=toughness:
            toWound_P = 1/6
        elif toughness*2<=strength:
            toWound_P = 1-(1/6)
        elif strength>toughness:
            toWound_P = 1-(2/6)
        elif toughness>strength:
            toWound_P = 2/6
        elif toughness==strength:
            toWound_P = 1/2
            
        toResist_P = armor + ap 
        

        if invulnerable < toResist_P and invulnerable >1:
            toResist_P = 1 - ((invulnerable - 1)/6)
        else:
            
            toResist_P = 0 if toResist_P >= 7 else 1-((toResist_P - 1)/6)

        hits,pHit = successChance(nAttacker, threshold,toHit_P, multiplier=attacks)
        if toHitReRoll == True:
            
            hits2,pHit2 = successChance((nAttacker*attacks)-hits, threshold,toHit_P)
            hits = hits+hits2
            pHit = (pHit+pHit2)/2 #Averaged probability. Still figuring out how to model this probabilistic case since I just recently dusted off my statistics and probability knowledge.
        
        iWounds,pWound = successChance(hits,threshold,toWound_P)
        
        if toWoundReRoll == True:
            
            iWounds2,pWound2 = successChance(hits-iWounds, threshold,toHit_P)
            iWounds = iWounds+iWounds2
            pWound = (pWound+pWound2)/2 #Averaged probability. Still figuring out how to model this probabilistic case since I just recently dusted off my statistics and probability knowledge.
        
        ,prWounds = successChance(iWounds,threshold,toResist_P)

        totalWounds = iWounds - rWounds
        totalWounds = 0 if totalWounds < 0 else totalWounds
        
        kills = totalWounds
        
        if wounds>1 and wounds>damage:
            
            kills = [wounds for wound in range(1,totalWounds+1)]

            for i in range(len(kills)):
                
                while kills[i] > 0 and totalWounds>0:
                    kills[i] -= damage
                    totalWounds-= 1
            kills = np.array(kills)
            kills = len(kills[kills <= 0])
                            
                
        return (kills, pHit,hits, pWound,iWounds, prWounds,rWounds)        
    

def showResults(kills, pHit,hits, pWound, wounds, prWounds, rWounds):
    
    print(str(kills)+" enemy kills with a probability of "+str(pHit)+ " to hit "+str(hits)+" times "
      " a probability "+str(pWound)+" to make "+str(wounds)+" wounds"+
      " and a probability of "+str(prWounds)+" to resist "+str(rWounds)+" wounds")
        

In [ ]:
np.random.seed(42)
sns.set()
n_bloodletter_successes = np.random.binomial(30, 1-(1/6), size=1000)

successes = np.sum(n_bloodletter_successes>= 25)

print(successes/1000)
print(cbdp(30,30,0.5))

bins = np.arange(0, max(n_bloodletter_successes)+1.5)-0.5

plt.hist(n_bloodletter_successes,normed=True, bins = bins)
_= plt.xlabel('Number of successes')
_= plt.ylabel('Binomial distribution')

plt.show()

plt.clf()

x,y = ecdf(n_bloodletter_successes)
plt.plot(x,y, marker='.', linestyle='none')
plt.show()

In [334]:
#Bloodletter attacking marine unit with unstopabble ferocity and murderous horde abilities
kills, pHit,hits, pWound,wounds, prWounds, rWounds = enemyUnitsEliminatedByAttackerUnitWithMelee(30, 2, 5,2,4, 3, ap=3,
                                       attacks=2, damage=1, threshold=0.9)
showResults(kills, pHit,hits, pWound,wounds, prWounds, rWounds)

kills, pHit,hits, pWound,wounds, prWounds, rWounds = enemyUnitsEliminatedByAttackerUnitWithMelee(30, 2, 4,1,4, 3,
                                                                                     threshold=0.9,toWoundReRoll=True)
showResults(kills, pHit,hits, pWound,wounds, prWounds, rWounds)

kills, pHit,hits, pWound,wounds, prWounds, rWounds = enemyUnitsEliminatedByAttackerUnitWithMelee(12, 3, 6,18,7, 3,invulnerable=3,ap=1,
                                                                                     damage=1,threshold=0.9)
showResults(kills, pHit,hits, pWound,wounds, prWounds, rWounds)

To resis P calculated: 0.16666666666666663
Probability to resist0.16666666666666663
12 enemy kills with a probability of 0.935219624834 to hit 46 times  a probability 0.902113312258 to make 27 wounds and a probability of 0.960690264961 to resist 2 wounds
To resis P calculated: 0.6666666666666667
Probability to resist0.6666666666666667
9 enemy kills with a probability of 0.949434441846 to hit 22 times  a probability 0.932062419177 to make 18 wounds and a probability of 0.956651936393 to resist 9 wounds
Invulnerable save before calculation3
To resis P before calculated: 4
Invulnerable save after calculation3
To resis P after calculated: 4
Probability of success0.3333333333333333
Hits1
Current Probability0.263374485597
Probability to resist4
Probability of success4
Hits1
Current Probability4
0 enemy kills with a probability of 0.933554242145 to hit 6 times  a probability 0.263374485597 to make 1 wounds and a probability of 4 to resist 1 wounds


In [314]:
cbdp(6,1,0.333333)

1